# Extract, Transform, Load (ETL) Project
For this project, an international financial analysis company tracks stock prices, commodities, forex rates, inflation rates.  My job is to extract financial data from various sources like websites, APIs and files provided by various financial analysis firms.
## Project Tasks
*   Collect data using APIs
*   Collect data using webscraping.
*   Read csv and json file types.
*   Extract data from the above file types.
*   Transform data.
*   Use the built in logging module.
*   Save the transformed data in a ready-to-load format.

# First phase - Webscraping


In this part I will Use webscraping to get bank information

## Imports

In [3]:
from bs4 import BeautifulSoup
import html5lib
import requests
import pandas as pd

## Extract Data Using Web Scraping


The wikipedia webpage https://web.archive.org/web/20200318083015/https://en.wikipedia.org/wiki/List_of_largest_banks provides information about largest banks in the world by various parameters. This part scrape the data from the table 'By market capitalization' and store it in a JSON file.


### Webpage Contents

In [4]:
url="https://web.archive.org/web/20200318083015/https://en.wikipedia.org/wiki/List_of_largest_banks"
html_data=requests.get(url).text

### Scraping the Data

Using BeautifulSoup parse the contents of the webpage.


In [5]:
soup=BeautifulSoup(html_data,'html.parser')

Load the data from the `By market capitalization` table into a pandas dataframe.

In [6]:
data = pd.DataFrame(columns=["Name", "Market Cap (US$ Billion)"])

for row in soup.find_all('tbody')[2].find_all('tr'):
    col = row.find_all('td')
    
    if (col != []):
        name = col[1].text.strip()
        market_cap = col[2].text.strip()
        data = data.append({"Name":name, "Market Cap (US$ Billion)":market_cap}, ignore_index=True)

In [7]:
data.head()

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399


### Loading the Data

In [9]:
data.to_json("bank_market_cap_1.json")

# Secend phase - Extract API Data

## Objectives

In this part I will collect exchange rate data using an API and store the data as a CSV.

In [4]:
#!mamba install pandas==1.3.3 -y
#!mamba install requests==2.26.0 -y

## Imports

In [1]:
import requests
import pandas as pd
import json

## Extract Data Using an API


Using ExchangeRate-API I will extract currency exchange rate data.

### Call the API

In [7]:
url = "https://api.apilayer.com/exchangerates_data/latest?base=EUR&apikey=8Aa39u0wcJ0I0rDXVaJxot8wpNWHmzsg"
response = requests.get(url)
result = json.loads(response.text)

### Save as DataFrame

In [8]:
df=pd.DataFrame(result)

In [9]:
# Drop unnescessary columns
df1=df[["rates"]]
df1.head()

,rates
AED,4.010772
AFN,94.453902
ALL,113.016463
AMD,424.064249
ANG,1.969453


### Load the Data

In [ ]:
# Save the Dataframe
df1.to_csv("exchange_rates_1.csv")

# Final phase - ETL

In this part I will:

*   Run the ETL process
*   Extract bank and market cap data from the JSON file
*   Transform the market cap currency using the exchange rate data
*   Load the transformed data into a seperate CSV

In [1]:
#!mamba install pandas==1.3.3 -y
#!mamba install requests==2.26.0 -y

## Imports.

In [26]:
import os
import glob
import pandas as pd
from datetime import datetime

## Extract


### JSON Extract Function

In [27]:
def extract_from_json(file_to_process): #This function will extract JSON files.
    dataframe = pd.read_json(file_to_process)
    return dataframe

## Extract Function

In [28]:
columns=['Name','Market Cap (US$ Billion)']

In [29]:
def extract():
    extracted_data=pd.DataFrame(columns=columns)
    for jsonfile in glob.glob("bank_market_cap_1.json"):
        extracted_data=pd.concat([extracted_data,extract_from_json(jsonfile)], ignore_index=True)
    return extracted_data

## Transform

transform function do:

1.  Changes the `Market Cap (US$ Billion)` column from USD to output_currency
2.  Rounds the Market Cap (US$ Billion)\` column to 3(dec_pnt) decimal places
3.  Rename `Market Cap (US$ Billion)` to `Market Cap (output_currency$ Billion)`

In [30]:
def transform(data,dec_pnt, output_currency):
    exchange = pd.read_csv('exchange_rates_1.csv', index_col = 0)
    exchange_rate = exchange.loc[output_currency,"rates"]
    data['Market Cap (US$ Billion)']=round(data['Market Cap (US$ Billion)'] *exchange_rate,dec_pnt)
    data_transfer=data.rename(columns={"Market Cap (US$ Billion)": "Market Cap (" + output_currency + "$ Billion)"})
    return data_transfer

## Load
This function takes a dataframe and load it to a csv named `bank_market_cap_gbp.csv`.

In [31]:
def load(target_file,data_to_load):
    data_to_load.to_csv(target_file, index=False)

## Logging Function


In [32]:
def log(message):
    # Write your code here
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

## Running the ETL Process


In [33]:
log("ETL Job Started")
log("Extract phase Started")
extracted_data=extract()
log("Extract phase Ended")
log("Transform phase Started")
transform_data=transform(extracted_data, 3, "GBP")
log("Transform phase Ended")
log("Load phase Started")
load("bank_market_cap_gbp.csv", transform_data)
log("Load phase Ended")
log("ETL Job Ended")

In [34]:
extracted_data.head()

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,342.356
1,Industrial and Commercial Bank of China,302.317
2,Bank of America,284.905
3,Wells Fargo,269.739
4,China Construction Bank,225.414


In [35]:
transform_data.head()

,Name,Market Cap (GBP$ Billion)
0,JPMorgan Chase,342.356
1,Industrial and Commercial Bank of China,302.317
2,Bank of America,284.905
3,Wells Fargo,269.739
4,China Construction Bank,225.414


In [36]:
with open('logfile.txt') as f:
    contents = f.read()
    print(contents)

2023-Apr-05-13:20:28,ETL Job Started
2023-Apr-05-13:20:28,Extract phase Started
2023-Apr-05-13:20:29,Extract phase Ended
2023-Apr-05-13:20:29,Transform phase Started
2023-Apr-05-13:20:29,Transform phase Ended
2023-Apr-05-13:20:29,Load phase Started
2023-Apr-05-13:20:29,Load phase Ended
2023-Apr-05-13:20:29,ETL Job Ended



## Authors


Mohammad Mohammadi